# Reverb.com Effects Pedals Exploratory Data Analysis (Univariate)

This analysis is of a sample of guitar pedals from [Reverb.com's](https://reverb.com/) effects pedals category. The data was retrieved using their all listings API. All together, there are 6,650 effects pedals in the CSV file I created with a separate Python script. Most of the pedals are from major pedal brands such as Dunlop, MXR, Boss, Electro-Harmonix, and several others.

More info on their Reverb.com APIs can be found here: https://dev.reverb.com/docs

I use [Plotly offline](https://plot.ly/python/offline/) for visualizations and interactivity. For data manipulation, I used [Pandas](http://pandas.pydata.org/) and Python's [re](https://docs.python.org/3.5/library/re.html) package. This notebook contains only univariate exploratory data analysis.

## Getting Started

First, I import all of the required packages and the CSV file created separately. In total, there are 31 variables and 6,650 pedals. It has 6,650 rows, and 32 columns.

In [1]:
import pandas as pd
import re
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [2]:
reverb_df = pd.read_csv('reverb_effects_pedals_v4_08032016.csv')
reverb_df.shape

(6650, 32)

Next I print out the first 3 rows to see how the DataFrame is structured.

In [3]:
reverb_df.head(3)

,category_uuids,condition,created_at,description,finish,has_inventory,id,inventory,listing_currency,make,...,shipping.us_rate.amount,shipping.us_rate.currency,shipping.us_rate.display,shipping.us_rate.symbol,shop_name,status_online,title,total_brand_inventory,web_url,year
0,"['c6602a28-e2e7-4e70-abeb-0fa38b320be6', '1580...",Brand New,2015-02-04T11:05:26-06:00,This great chorus pedal is designed specifical...,NaN,True,474785,1,USD,Boss,...,0.0,USD,$0,$,Music Toyz,live,Boss CEB-1 Bass Chorus,4571,https://reverb.com/item/474785-boss-ceb-1-bass...,NaN
1,"['c6602a28-e2e7-4e70-abeb-0fa38b320be6', '1580...",Brand New,2016-04-13T17:20:59-05:00,Boss CEB-3 Bass ChorusThe CEB-3 Bass Chorus pe...,NaN,True,2067829,1,USD,Boss,...,0.0,USD,$0,$,Pro Guitar Shop,live,Boss CEB-3 Bass Chorus,4571,https://reverb.com/item/2067829-boss-ceb-3-bas...,NaN
2,"['3b09f948-3462-4ac2-93b3-59dd66da787e', '1738...",Brand New,2013-04-24T20:02:13-05:00,One of the most beloved echo effects ever made...,NaN,True,5712,11,USD,Boss,...,0.0,USD,$0,$,Chicago Music Exchange,live,Boss RE-20 Space Echo,4571,https://reverb.com/item/5712-boss-re-20-space-...,NaN


Since some of the labels are cut off in the notebook, I print out all of the DataFrame columns to see all of the possible variables in the dataset.

In [4]:
reverb_df.columns

Index(['category_uuids', 'condition', 'created_at', 'description', 'finish',
       'has_inventory', 'id', 'inventory', 'listing_currency', 'make', 'model',
       'offers_enabled', 'price.amount', 'price.currency', 'price.display',
       'price.symbol', 'price_drop.display', 'price_drop.percent',
       'published_at', 'shipping.local', 'shipping.us', 'shipping.us_rate',
       'shipping.us_rate.amount', 'shipping.us_rate.currency',
       'shipping.us_rate.display', 'shipping.us_rate.symbol', 'shop_name',
       'status_online', 'title', 'total_brand_inventory', 'web_url', 'year'],
      dtype='object')

I noticed there is an `id` column, which I believe is used as a unique identifier for each pedal. To make sure I don't have duplicates in the DataFrame, I printed out the value counts of each id. Turns out, there are multiple instances of the same id for multiple pedals. Below are a couple of duplicate ids, each with 4 instances in the dataset.

In [5]:
reverb_df['id'].value_counts()[0:3]

1363704    4
2159715    4
2003513    4
Name: id, dtype: int64

To make sure these are actually duplicate pedals, I printed out the URL for one of the duplicate ids to see if each pedal had the same web url. Each of them did for id 1363704.

In [6]:
reverb_df[reverb_df.id == 1363704].web_url

3091    https://reverb.com/item/1363704-mooer-ninety-o...
3101    https://reverb.com/item/1363704-mooer-ninety-o...
3366    https://reverb.com/item/1363704-mooer-ninety-o...
3486    https://reverb.com/item/1363704-mooer-ninety-o...
Name: web_url, dtype: object

Because of this, I decided to drop any duplicate rows based on the web URL, while keeping the first duplicate instance. This led to 1129 pedals being removed from the dataset.

In [7]:
reverb_df = reverb_df.drop_duplicates('web_url', keep='first')

In [8]:
reverb_df.shape

(5521, 32)

Now that all duplicate pedals have been removed, I decided to make a simple bar chart to show how many pedals by brand were in the dataset. Below I print out all of the pedal brands in the CSV file. Some of the brands appear to be duplicated by capitalization, casing errors and other text-based issues. Others appear to contain more than one brand name, maybe some type of hybrid pedal between 2 brands.

In [9]:
reverb_df.make.unique()

array(['Boss', 'Electro-Harmonix', 'MXR', 'Dunlop', 'Dunlop Accessories',
       'EarthQuaker Devices', 'TC Electronic', 'Mooer', 'Mooer Juicer',
       'Mooer Ana Echo & Hustle Drive Two Pedals', 'Unknown', 'Line 6',
       'Molten Voltage', 'Zvex', 'Wampler', 'Sale', 'DigiTech',
       'Digitech Hpro', 'JHS', 'Jet City', 'Other', 'Ibanez',
       'Ibanez AD-9 Analog Delay With Keeley Mod', 'Keeley',
       'Alchemy Audio', 'Roland', 'Sheet Music', 'DD-500', 'Acoustic',
       'Ph-3', 'Analogman', "Custom Badass '78 Distortion", 'EVH',
       'Dunlop Effects', 'MXR EVH Flanger Pedal',
       'MXR Phase 90 1979 BLOCK LOGO!', 'Dunlop Manufacturing',
       'Dunlop CBM95 CryBaby Mini Wah Effects Pedal - FREE Shipping',
       'Dunlop Gcb-95', 'Rukavina', 'Mission Engineering', '******ZVEX',
       'wampler', 'WAMPLER', 'Ernie Ball', 'JHS Effects', 'ProCo',
       'JHS/Keeley', 'Ibanez/ Maxon', 'Ibanez (maxon)', 'Ibanez TS-808',
       'Ibanez Compressor'], dtype=object)

To make things simple, I write a function below to update the `make` column of the `reverb_df` DataFrame by consolidating any duplicates based on string matching. Any pedals containing more than one brand in the name were also consolidated.

In [10]:
def fix_make_names(row):
    r = row
    if 'Ibanez' in r:
        return 'Ibanez'
    elif 'WAMPLER' in r or 'wampler' in r:
        return 'Wampler'
    elif 'Mooer' in r:
        return 'Mooer'
    elif 'Dunlop' in r:
        return 'Dunlop'
    elif 'MXR' in r:
        return 'MXR'
    elif 'JHS' in r:
        return 'JHS'
    elif 'ZVEX' in r:
        return 'Zvex'
    return r

After applying the function and printing out the value counts of each brand, it looks like the problem has been solved. However, I noticed there are several brands with less than 20 pedals in the dataset, such as Ernie Ball and EVH.

In [11]:
reverb_df.make = reverb_df.make.apply(fix_make_names)

In [12]:
reverb_df.make.value_counts()

Dunlop                          595
Ibanez                          589
MXR                             535
DigiTech                        520
Boss                            519
Zvex                            511
Wampler                         496
JHS                             471
Mooer                           443
Electro-Harmonix                202
TC Electronic                   200
EarthQuaker Devices             194
Line 6                          186
Roland                           15
Unknown                          13
Rukavina                          3
EVH                               3
Alchemy Audio                     3
Sheet Music                       3
Jet City                          3
Keeley                            3
Acoustic                          3
Ernie Ball                        1
Sale                              1
Ph-3                              1
ProCo                             1
Mission Engineering               1
Other                       

I decided to exclude any brands with less than 100 pedals from this project for now. Below, I use the Pandas `groupby` method to group by pedal brand, and then filter out any brands that have less than 100 observations in the dataset. This removes 60 pedals from our DataFrame.

In [13]:
reverb_df = reverb_df.groupby('make').filter(lambda x: len(x) > 100)
reverb_df.shape

(5461, 32)

## Number of Pedals by Brand

Below I use Plotly to plot a simple pedal brand count to see the number of pedals by brand in this dataset. What's interesting about Plotly's histogram object is that if you pass a categorical x value to it, it will plot a bar plot instead of a histogram. Makes things easy! I decided to write a function to avoid rewriting the histogram plot code  multiple times throughout the notebook.

Dunlop and Ibanez have the most pedals in the dataset (over 580 each).  

In [14]:
def plot_histogram(x, xaxis_title, yaxis_title, chart_title, xaxis_tickfont_size=12, yaxis_tickfont_size=12):
    data = [Histogram(x=x)]
    layout = Layout(
        xaxis = dict(title=xaxis_title, tickfont=dict(size=xaxis_tickfont_size)),
        yaxis = dict(title=yaxis_title, tickfont=dict(size=yaxis_tickfont_size)),
        title = chart_title
    )
    fig = Figure(data=data, layout=layout)
    return fig

In [15]:
iplot(plot_histogram(reverb_df.make, None, 'Number of Pedals', 'Number of Pedals by Brand'))

## Total Condition Counts

Next I want to see what kind of condition the pedals are in. Conditions range from poor, to brand new, which can be seen using the Pandas `unique` method on the `condition` column. I was pretty surprised at how skewed it was towards mostly brand new pedals (about 74%). However I'm not entirely sure what the difference is between some of these conditions. For example, the difference between excellent and very good is difficult to decipher without further research.

In [16]:
reverb_df.condition.unique()

array(['Brand New', 'Excellent', 'Mint', 'Very Good', 'B-Stock', 'Good',
       'Fair', 'Non Functioning', 'Poor'], dtype=object)

In [17]:
iplot(plot_histogram(reverb_df.condition, 'Condition Type', 'Number of Pedals', 
                     'Number of Pedals by Condition Type'))

Out of curiosity I wanted to see what 2 pedals were classified as non-functioning. One is a Dunlop Cry Baby, which is a very popular wah pedal. The other is a Digitech Preamp Processor. After looking at their pages online, turns out both have already been sold! 

In [18]:
reverb_df[reverb_df.condition == 'Non Functioning'].title

2190              Dunlop Cry baby vintage ? black
4934    DigiTech GSP 2101 Studio Preamp Processor
Name: title, dtype: object

## Shop Name Counts

Next, I wanted to see what stores sold the most pedals to get an idea what stores sold on Reverb.com. To do this, I create a new DataFrame based on the number of times each store is listed and then sort by the name of the shop.

In [19]:
shop_name_counts = pd.DataFrame(reverb_df.shop_name.value_counts()).reset_index().sort_values(by='shop_name')
shop_name_counts.columns = ['shop_name', 'shop_count']
shop_name_counts.head(3)

,shop_name,shop_count
513,Bigfoot Music,1
676,The Mothership UK,1
677,Nic's Gear Locker,1


About 64% of the stores only sell one pedal in this dataset. This is probably due to the way the data was collected, but still is interesting.

In [20]:
def count_shops(shop_counts):
    shop_vals = list(shop_counts)
    count = 0
    for x in shop_vals:
        if x == 1:
            count += 1
    print(round(count / len(shop_vals), 2))
count_shops(reverb_df.shop_name.value_counts())

0.64


I create a bar plot function for the same reasons as the histogram function above. I use a bar plot here because I'm using the `shop_count` column as the y-axis variable, kind of like how you can declare the y variable as the `identity` variable in ggplot2. 

In [21]:
def plot_bar(x, y, xaxis_title, yaxis_title, chart_title, xaxis_tickfont_size=12, yaxis_tickfont_size=12):
    
    data = [Bar(x=x, y=y)]
    layout = Layout(
        xaxis = dict(title=xaxis_title, tickfont=dict(size=xaxis_tickfont_size)),
        yaxis = dict(title=yaxis_title, tickfont=dict(size=yaxis_tickfont_size)),
        title = chart_title
    )
    fig = Figure(data=data, layout=layout)
    return fig

Below is a plot including all shops in the dataset. After looking at this closely for a bit, I found out that some larger MI retailers like Sam Ash sell on Reverb.com, which I didn't initially realize.

In [22]:
iplot(plot_bar(shop_name_counts.shop_name, shop_name_counts.shop_count, None, 'Number of Pedals', 
               'Number of Pedals by Shop', 6))

The top seller is [Pro Guitar Shop](https://proguitarshop.com/), with 316 pedals. Second is [Chicago Music Exchange](http://www.chicagomusicexchange.com/), which has a great vintage collection and product content. Third is [Gear Tree](https://www.geartree.com/).

In [23]:
reverb_df.shop_name.value_counts()[:3]

Pro Guitar Shop           316
Chicago Music Exchange    301
GearTree                  233
Name: shop_name, dtype: int64

The bar plot above is very hard to read. To get a closer look on the top stores, I create a new DataFrame that only contains the top 25 stores based on the number of pedals sold.

In [24]:
top_shops = shop_name_counts.sort_values(by='shop_count').tail(25)

In [25]:
iplot(plot_bar(top_shops.shop_name, top_shops.shop_count, None, 'Number of Pedals', 
               '# of Pedals Sold by Shop (Top 25 Stores)', 8))

## Number of Pedals by Year

Next, I wanted to plot the number of pedals by what year they were created. The `year` column contains a bunch of invalid information. For this project, I decided that only the pedals with a valid year after 1900 would be included for this project. Below are the unique values in the column without any cleaning.

In [26]:
reverb_df.year.unique()

array([nan, '2007', '2016', '2013', '1975', '5150', 'New', '2014', '2015',
       '2001', "2000's", '1500', '2012', '4543', 'Reissue', '1981', '1984',
       "1980's", '0851', '1979', '90s', '1985', '1997', '0550', '0390',
       '9835', '0139', "1990's", '9808', '7757', '1970-1980',
       '2010 - 2015', '1983', '1996', '2002', '0219', '7033', '1987',
       '80s', '1986', "2010's", '1980s', '2010s', '1989', '2008',
       '80s/90s', '1982', '0180', 'recent', '2010', '9900', '2000s',
       '0676', '0143', '9561', '9266', '7579', '8071', '0579', "1980's ",
       '0886', '0970', "80's", '1980', '????', '1979-83', '2000', '1978',
       'c 1979', "70's", '1977', '0831', '??', 'c 1975', 'Vintage ',
       '0937', "70-80's", '0790', '1999', '2013 or Later', '1995', '2005',
       '8123', '7851', '2003', '1990', '?', '0304', '0050', '1996-1997',
       ' ', '4977', '4976', '8665', '2011', '0685', '0284', '8379', '0774',
       '0588', '9848', '0554', '0316', '0791', '0856', '0306', '0840'

I create the `fix_years` function below to match a regular expression to each row. It matches any 4 digit number starting with 19 or 20 to account for the 20th and 21st century.

In [27]:
def fix_years(row):
    if re.search('^(19|20)\d{2}$', str(row)):
        return row
    return None

After applying the function, only the years from the late 1970's to 2016 are included, which is what I want.

In [28]:
reverb_df.year = reverb_df.year.apply(fix_years)
reverb_df.year.unique()

array([None, '2007', '2016', '2013', '1975', '2014', '2015', '2001',
       '2012', '1981', '1984', '1979', '1985', '1997', '1983', '1996',
       '2002', '1987', '1986', '1989', '2008', '1982', '2010', '1980',
       '2000', '1978', '1977', '1999', '1995', '2005', '2003', '1990',
       '2011', '2004', '2006', '2009', '1993', '1976', '1988', '1992'], dtype=object)

The majority of pedals were created in the late 2000's, which may be why most pedals are considered "brand new". The other large group of pedals originates from the late 1970's into the 1980's. 

In [29]:
iplot(plot_histogram(reverb_df.year, 'Year', 'Number of Pedals', 'Number of Pedals by Year of Origin'))

## # of Pedals by Currency Listing

About 97% of pedals are being sold in U.S. dollars, with some others being sold for Euros, the English pound, Canadian, and Austrailian dollars.

In [30]:
iplot(plot_histogram(reverb_df.listing_currency, 'Currency', 'Number of Pedals', 'Number of Pedals by Currency'))

The next few plots will be only focusing on pedals being sold for U.S. dollars, since that is the most common currency used in this dataset.

## Number of Pedals by Price Amount

Since I only want to focus on USD, I create a new DataFrame only containing pedals being sold for U.S. dollars.

In [31]:
us_currency = reverb_df[reverb_df.listing_currency == 'USD']

Prices range from 10 dollars, to over 1300 dollars for a single pedal, with a mean price of approximately 164 dollars. However, there's a couple of outliers above 600 dollars that may be skewing that metric, which can be seen in the histogram below.

In [32]:
us_currency['price.amount'].describe()

count    5317.000000
mean      164.002005
std        93.475861
min        10.000000
25%        99.000000
50%       149.000000
75%       199.970000
max      1399.000000
Name: price.amount, dtype: float64

In [33]:
iplot(plot_histogram(us_currency['price.amount'], 'Price ($)', '# of Pedals', '# of Pedals by Price ($)'))

Cutting out all pedals past the 75th perecentile lowers the price standard deviation by 48 dollars, which makes more sense to me.

In [34]:
price_no_outliers = us_currency[us_currency['price.amount'] < 200]

In [35]:
price_no_outliers['price.amount'].describe()

count    4129.000000
mean      126.262502
std        45.663689
min        10.000000
25%        89.990000
50%       122.090000
75%       160.000000
max       199.990000
Name: price.amount, dtype: float64

Lots of spikes in pricing, however the majority of pedals are either in the 100, 150, or 200 dollar range.

In [36]:
iplot(plot_histogram(price_no_outliers['price.amount'], 'Price ($)', 'Number of Pedals', 
                     'Number of Pedals by Price ($)'))

The most expensive pedal in the dataset is a Line 6 Helix, which helps players model over 150 guitar amps, includes 12 foot switches, and can be used to create custom sounding rigs from a single pedalboard.

In [37]:
reverb_df.ix[reverb_df['price.amount'].idxmax()]

category_uuids                        ['fa10f97c-dd98-4a8f-933b-8cb55eb653dd']
condition                                                                 Mint
created_at                                           2016-07-21T00:12:32-05:00
description                  ATTENTION; I WILL BE OUT OF THE COUNTRY AUGUST...
finish                                                                    2015
has_inventory                                                              NaN
id                                                                     2585046
inventory                                                                    1
listing_currency                                                           USD
make                                                                    Line 6
model                                                                    HELIX
offers_enabled                                                            True
price.amount                                        

The cheapest pedal is another Line 6 pedal; the Pod HD500. This one is being sold for 34% off. I read the description online and it turns out that this is only a set of sound patches being sold, not the actual pedalboard itself, which explains why it's so cheap. 

In [38]:
reverb_df.ix[reverb_df['price.amount'].idxmin()]

category_uuids                        ['2d6093b4-6b33-474e-b07c-25f6657d7956']
condition                                                            Brand New
created_at                                           2014-12-02T21:02:46-06:00
description                  Up for sale is a collection of Line 6 HD500 pa...
finish                                                                     NaN
has_inventory                                                             True
id                                                                      390869
inventory                                                                    6
listing_currency                                                           USD
make                                                                    Line 6
model                                                                POD HD500
offers_enabled                                                             NaN
price.amount                                        

## Number of Pedals by Price Drop Percentage

Approximately 11% of pedals have some type of price drop, ranging from 1% to over 68%. This histogram includes all currency types, not just U.S. dollars. 

In [39]:
round(1 - (reverb_df[reverb_df['price_drop.percent'].isnull()].shape[0] / reverb_df.shape[0]), 2)

0.11

In [40]:
iplot(plot_histogram(reverb_df['price_drop.percent'], 'Price Drop (%)', 'Number of Pedals', 
                     'Number of Pedals by % Price Drop'))

## Local Shipping Costs

Several outliers can be seen in U.S. shipping costs for pedals. The maxiumum shiping cost in the dataset is $100, which is very high in my opinion.

In [41]:
us_currency['shipping.us_rate.amount'].describe()

count    5304.000000
mean        2.997353
std         6.356127
min         0.000000
25%         0.000000
50%         0.000000
75%         3.990000
max       100.000000
Name: shipping.us_rate.amount, dtype: float64

After cutting out some larger outliers, the mean shipping cost is about 12 dollars, with  a standard deviation of about 7 dollars.

In [42]:
us_shipping_amount_clean = us_currency[(us_currency['shipping.us_rate.amount'] > 0) & \
                                       (us_currency['shipping.us_rate.amount'] <= 60)]

In [43]:
us_shipping_amount_clean['shipping.us_rate.amount'].describe()

count    1359.000000
mean       11.565828
std         6.849078
min         0.600000
25%         7.995000
50%        10.000000
75%        14.990000
max        60.000000
Name: shipping.us_rate.amount, dtype: float64

The most common shipping cost is in the 10 dollar range, with other spikes at 15, 20 and a few other multiples of 5.

In [44]:
iplot(plot_histogram(us_shipping_amount_clean['shipping.us_rate.amount'], 
                     'Shipping Cost ($)', '# of Pedals', 
                     '# of Pedals by Shipping Cost ($)'))

The pedal that costs 100 dollars to ship a Line 6 POD HD500X. From the description online, it is currently in San Francisco at a store called Gear Emporium, a b-stock (although labeled as "very good" condition) and has 1 offer from a buyer. It's very interesting that the most expensive, the least expensive, and the highest shipping amount have all been Line 6 pedals. Could be coincidence, but something to look into later.

In [45]:
us_currency.ix[us_currency['shipping.us_rate.amount'].idxmax()]

category_uuids                        ['2d6093b4-6b33-474e-b07c-25f6657d7956']
condition                                                            Very Good
created_at                                           2016-06-09T17:10:44-05:00
description                  I just purchased this unit from Reverb one mon...
finish                                                                   Black
has_inventory                                                              NaN
id                                                                     2357195
inventory                                                                    1
listing_currency                                                           USD
make                                                                    Line 6
model                                                               POD HD500X
offers_enabled                                                            True
price.amount                                        

Bivariate and multivariate analysis to come soon...